In [37]:
from Explanation.utils import *
from Explanation.SJsrc import *
from Explanation.HKUSTsrc import *

In [27]:
import sys
class ParseConfigFile(argparse.Action):

    def __call__(self, parser, namespace, filename, option_string=None):

        if not os.path.exists(filename):
            raise ValueError('cannot find config at `%s`'%filename)

        with open(filename) as f:
            config = json.load(f)
            for key, value in config.items():
                setattr(namespace, key, value)
                
def parse_args():
    parser = argparse.ArgumentParser()

    # model
    parser.add_argument('--model_name', default='NRSR')
    parser.add_argument('--model_path', default='.\parameter')
    parser.add_argument('--num_relation', type= int, default=102)
    parser.add_argument('--d_feat', type=int, default=6)
    parser.add_argument('--hidden_size', type=int, default=128)
    parser.add_argument('--num_layers', type=int, default=2)
    parser.add_argument('--dropout', type=float, default=0.1)
    parser.add_argument('--K', type=int, default=1)
    parser.add_argument('--loss_type', default='')
    parser.add_argument('--config', action=ParseConfigFile, default='')
    # for ts lib model
    parser.add_argument('--seq_len', type=int, default=60)
    parser.add_argument('--moving_avg', type=int, default=21)
    parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder')
    parser.add_argument('--embed', type=str, default='timeF',
                        help='time features encoding, options:[timeF, fixed, learned]')
    parser.add_argument('--freq', type=str, default='b',
                        help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
    parser.add_argument('--distil', action='store_false',
                        help='whether to use distilling in encoder, using this argument means not using distilling',
                        default=False)
    parser.add_argument('--factor', type=int, default=1, help='attn factor')
    parser.add_argument('--n_heads', type=int, default=1, help='num of heads')
    parser.add_argument('--d_ff', type=int, default=64, help='dimension of fcn')
    parser.add_argument('--activation', type=str, default='gelu', help='activation')
    parser.add_argument('--e_layers', type=int, default=8, help='num of encoder layers')
    parser.add_argument('--top_k', type=int, default=5, help='for TimesBlock')
    parser.add_argument('--pred_len', type=int, default=-1, help='the length of pred squence, in regression set to -1')
    parser.add_argument('--de_norm', default=True, help='de normalize or not')

    # data
    parser.add_argument('--data_set', type=str, default='csi360')
    parser.add_argument('--target', type=str, default='t+0')
    parser.add_argument('--pin_memory', action='store_false', default=True)
    parser.add_argument('--batch_size', type=int, default=-1)  # -1 indicate daily batch
    parser.add_argument('--least_samples_num', type=float, default=1137.0)
    parser.add_argument('--label', default='')  # specify other labels
    parser.add_argument('--start_date', default='2019-01-01')
    parser.add_argument('--end_date', default='2019-01-05')

    # input for csi 300
    parser.add_argument('--data_root', default='.\Data')
    parser.add_argument('--market_value_path', default= '.\Data\csi300_market_value_07to20.pkl')
    parser.add_argument('--stock2concept_matrix', default='.\Data\csi300_stock2concept.npy')
    parser.add_argument('--stock2stock_matrix', default='.\Data\csi300_multi_stock2stock_hidy_2024.npy')


    parser.add_argument('--stock_index', default='.\Data\csi300_stock_index_2024.npy')
    parser.add_argument('--model_dir', default='.\parameter')
    parser.add_argument('--events_files', default='.\event_data_sigle_stock.json')

    parser.add_argument('--overwrite', action='store_true', default=False)
    parser.add_argument('--device', default='cpu')
    parser.add_argument('--relation_name_list_file', default=r'.\Data\new_relation_name_list2024.json')
  
    filtered_args = [arg for arg in sys.argv if not arg.startswith('-f') and not arg.endswith('.json')]
    args = parser.parse_args(filtered_args[1:])  # Skip the script name
    return args

In [28]:
args = parse_args()
args.start_date = '2024-04-10'
args.end_date = '2024-04-11' # 输入查询日期
args.check_stock_list = None # 输入待查股票列表，如果输入None则查询CSI300所有股票
args.model_name =  'NRSR' #基于知识的股票预测模型
events_files = args.events_files

In [29]:
import json
with open(events_files, 'r', encoding='utf-8') as f: # 读入事件文件
    events_data = json.load(f) 

In [31]:
from Explanation.ExplanationInterface import run_input_gradient_explanation
args.explainer = 'inputGradientExplainer'
exp_result_dict, sorted_stock_rank, _ = run_input_gradient_explanation(args, events_data)
print(exp_result_dict) # 输出解释结果
print(sorted_stock_rank) # 股票排名

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.58it/s]


{'2024-04-10': {'SH600000': {'pred_result': -0.03337288, 'SZ001979': {'total_score': 0.5378007, 'relation': ['同行业'], 'events': {}}, 'SZ002142': {'total_score': 0.5145984, 'relation': ['同行业'], 'events': {'2024-02-12': ['35股获券商买入评级 宁波银行目标涨幅达105.63%']}}, 'SZ000001': {'total_score': 0.51399857, 'relation': ['股份制银行Ⅲ', '同行业'], 'events': {'2024-02-21': ['平安银行突然拉升涨停 该行投资者热线：无应披露而未披露信息', '平安银行涨停 成交额超35亿元', '平安银行巨量涨停 国泰君安上海分公司买入2.16亿元', '平安银行盘中涨停', '大金融板块持续走高 平安银行涨超8%', '平安银行行长特别助理蔡新发离职 公茂江拟任上海分行行长', '平安银行今日涨停 方新侠席位净买入1.31亿元', '平安银行回应涨停：目前无应披露而未披露信息', '银行股震荡拉升，中国银行、农业银行再创历史新高，平安银行、成都银行、齐鲁银行涨超2%'], '2024-02-19': ['平安银行变阵：架构改革路线图浮出水面 管理干部发生批量变动'], '2024-02-22': ['平安银行涨停领涨 银行股估值修复持续演绎']}}}, 'SH600009': {'pred_result': 0.034151025, 'SH601888': {'total_score': 0.9361988, 'relation': ['投资', '同行业', '合作', '供应'], 'events': {}}, 'SH601688': {'total_score': 0.5138409, 'relation': ['增加持股'], 'events': {}}, 'SH601211': {'total_score': 0.4987488, 'relation': ['合作'], 'events': {'2024-02-23': ['国泰君安：2024年钢铁行业需求有

In [32]:
from Explanation.ExplanationInterface import run_xpath_explanation
args.explainer = 'xpathExplainer'
exp_result_dict, sorted_stock_rank, _ = run_xpath_explanation(args, events_data, get_fidelity=False, top_k=3)
print(exp_result_dict)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]D:\ProgramFiles\Miniconda\envs\qlib\lib\site-packages\dgl\dataloading\dataloader.py:1149: DGLWarning: Dataloader CPU affinity opt is not enabled, consider switching it on (see enable_cpu_affinity() or CPU best practices for DGL [https://docs.dgl.ai/tutorials/cpu/cpu_best_practises.html])
  dgl_warning(
100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [03:42<00:00, 111.14s/it]


{'2024-04-10': {'SH600000': {'SH603019': {'total_score': 55, 'relations': ['合作'], 'events': {}}, 'SZ001979': {'total_score': 31, 'relations': ['同行业'], 'events': {}}, 'SZ002142': {'total_score': 12, 'relations': ['同行业'], 'events': {'2024-02-12': ['35股获券商买入评级 宁波银行目标涨幅达105.63%']}}}, 'SH600009': {'SH601888': {'total_score': 82, 'relations': ['投资', '同行业', '合作', '供应'], 'events': {}}, 'SH603799': {'total_score': 9, 'relations': ['上升'], 'events': {}}, 'SZ000333': {'total_score': 7, 'relations': ['同行业'], 'events': {'2024-02-19': ['北上资金今日净买入贵州茅台9.40亿元、美的集团4.23亿元']}}}, 'SH600010': {'SH601988': {'total_score': 41, 'relations': ['争议'], 'events': {}}, 'SH600019': {'total_score': 30, 'relations': ['板材', '合作'], 'events': {}}, 'SH603799': {'total_score': 27, 'relations': ['同行业'], 'events': {}}}, 'SH600011': {'SH600030': {'total_score': 42, 'relations': ['同行业', '争议'], 'events': {'2024-02-23': ['中信证券：Sora横空出世 关注三条投资主线', '中信证券：兼具成长性和业绩兑现能力的国内铜矿股票的估值水平具备提升潜力', '中信证券：Sora发布 关注算力+应用+AI监管', '中信证券：兼具成长性和业绩兑现能力

In [33]:
from Explanation.ExplanationInterface import run_gnn_explainer
args.explainer = 'gnnExplainer'
exp_result_dict, sorted_stock_rank, _ = run_gnn_explainer(args, events_data, top_k=3)
print(exp_result_dict)

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [17:04<00:00, 512.23s/it]


{'2024-04-10': {'SH600000': {'SH600000': {'total_score': 2.5647318363189697, 'relations': ['股份制银行Ⅲ', '同行业', '增加持股', '合作', '争议', '下降', '上升', '供应', '竞争'], 'events': {'2024-02-11': ['浦发银行全面推进落实城市房地产融资协调机制']}}, 'SH600036': {'total_score': 0.9683955907821655, 'relations': ['股份制银行Ⅲ', '同行业', '下降'], 'events': {'2024-02-14': ['招商银行H股涨5%领涨内银股'], '2024-02-21': ['北上资金今日净买入贵州茅台13.86亿元、招商银行7.71亿元，净卖出三花智控3.43亿元、长安汽车0.97亿元']}}, 'SH601988': {'total_score': 0.9572372436523438, 'relations': ['同行业', '增加持股', '争议'], 'events': {}}}, 'SH600009': {'SH600009': {'total_score': 2.108368396759033, 'relations': ['机场', '投资', '同行业', '增加持股', '合作', '下降', '上升', '供应'], 'events': {}}, 'SH601888': {'total_score': 0.9603880643844604, 'relations': ['投资', '同行业', '合作', '供应'], 'events': {}}, 'SH600276': {'total_score': 0.45386701822280884, 'relations': ['增加持股'], 'events': {}}}, 'SH600010': {'SH600010': {'total_score': 2.7585153579711914, 'relations': ['板材', '投资', '同行业', '增加持股', '合作', '争议', '下降', '上升', '优越的', '供应'], 'events': {}